![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Multimodales RAG - Retrieval Augmented Generation
</b></font> </br></p>

In [1]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint, install_packages
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

✓ OPENAI_API_KEY erfolgreich gesetzt
✓ HF_TOKEN erfolgreich gesetzt

Python Version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]

Installierte LangChain-Bibliotheken:
langchain                                0.3.27
langchain-community                      0.3.27
langchain-core                           0.3.74
langchain-experimental                   0.3.4
langchain-ollama                         0.3.6
langchain-openai                         0.3.30
langchain-text-splitters                 0.3.9

IP-Adresse: 35.196.195.212
Hostname: 212.195.196.35.bc.googleusercontent.com
Stadt: North Charleston
Region: South Carolina
Land: US
Koordinaten: 32.8546,-79.9748
Provider: AS396982 Google LLC
Postleitzahl: 29415
Zeitzone: America/New_York


In [2]:
#@title 🛠️ Installationen { display-mode: "form" }
install_packages([
    ('markitdown[all]', 'markitdown'),
    'langchain_chroma',
])

🔄 Installiere markitdown[all]...
✅ markitdown[all] erfolgreich installiert und importiert
🔄 Installiere langchain_chroma...
✅ langchain_chroma erfolgreich installiert und importiert


In [3]:
#@title 📂 Dokumente & Bilder { display-mode: "form" }
!rm -rf files
!mkdir files
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_1.txt -o files/biografien_1.txt
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_2.md -o files/biografien_2.md
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_3.pdf -o files/biografien_3.pdf
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/biografien_4.docx -o files/biografien_4.docx
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/a_retro-futuristic_robot_dall_e.jpg -o files/a_retro-futuristic_robot_dall_e.jpg
!curl -L https://raw.githubusercontent.com/ralf-42/GenAI/main/02%20data/Apfel.png -o files/Apfel.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5324  100  5324    0     0  27350      0 --:--:-- --:--:-- --:--:-- 27302
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7868  100  7868    0     0  47095      0 --:--:-- --:--:-- --:--:-- 47397
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 68184  100 68184    0     0   161k      0 --:--:-- --:--:-- --:--:--  162k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18743  100 18743    0     0  36925      0 --:--:-- --:--:-- --:--:-- 36968
  % Total    % Received % Xferd  Average Speed   Tim

# 1 | Funktionale Architektur
---



<p><font color='black' size="5">
Aufbau des funktionalen RAG-Systems
</font></p>

Das funktionale System besteht aus drei Hauptkomponenten:

**1. Konfiguration & Setup:**
- `RAGConfig`: Zentrale Konfiguration als Dataclass
- `RAGComponents`: Container für alle System-Komponenten
- `init_rag_system()`: Einmalige Initialisierung

**2. Dokumenten-Verarbeitung:**
- `add_text_document()`: Einzelnes Text-Dokument hinzufügen
- `add_image()`: Einzelnes Bild hinzufügen  
- `process_directory()`: Gesamtes Verzeichnis verarbeiten

**3. Suche & Abfrage:**
- `search_texts()`: Reine Text-Suche
- `search_images()`: Reine Bild-Suche
- `multimodal_search()`: Kombinierte Suche

**Datenfluss:**
```
Dokumente → Verarbeitung → Embeddings → Vektordatenbank → Suche → LLM → Antwort
```



# 2 | Implementierung
---



## 2.1 Importe und Grundkonfiguration

In [4]:
from pathlib import Path
from typing import List, Dict, Any, Tuple
import uuid
from dataclasses import dataclass

from markitdown import MarkItDown
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer
from PIL import Image
import chromadb

## 2.2 Konfiguration und Datenstrukturen

In [5]:
@dataclass
class RAGConfig:
    """Zentrale Konfiguration für das RAG-System"""
    chunk_size: int = 200
    chunk_overlap: int = 20
    text_threshold: float = 1.2
    image_threshold: float = 0.8
    clip_model: str = 'clip-ViT-B-32'
    text_model: str = 'text-embedding-3-small'
    llm_model: str = 'gpt-4o-mini'
    db_path: str = './multimodal_rag_db'

@dataclass
class RAGComponents:
    """Container für alle RAG-System-Komponenten"""
    text_embeddings: OpenAIEmbeddings
    clip_model: SentenceTransformer
    llm: ChatOpenAI
    text_splitter: RecursiveCharacterTextSplitter
    markitdown: MarkItDown
    chroma_client: chromadb.PersistentClient
    text_collection: Chroma
    image_collection: Any

## 2.3 System-Initialisierung

In [6]:
def init_rag_system(config: RAGConfig = RAGConfig()) -> RAGComponents:
    """Initialisiert alle RAG-Komponenten"""
    print(f"🚀 Initialisiere RAG-System in {config.db_path}")

    # KI-Modelle laden
    text_embeddings = OpenAIEmbeddings(model=config.text_model)
    print("✅ OpenAI Text-Embeddings initialisiert")

    print("🖼️ Lade CLIP-Modell...")
    clip_model = SentenceTransformer(config.clip_model)
    print("✅ CLIP-Modell geladen")

    llm = ChatOpenAI(model=config.llm_model, temperature=0)

    # Text-Verarbeitung
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config.chunk_size,
        chunk_overlap=config.chunk_overlap
    )
    markitdown = MarkItDown()

    # Datenbank einrichten
    Path(config.db_path).mkdir(exist_ok=True)
    chroma_client = chromadb.PersistentClient(path=config.db_path)

    # Text-Collection
    text_collection = Chroma(
        collection_name="texts",
        embedding_function=text_embeddings,
        persist_directory=config.db_path
    )

    # Bild-Collection
    collections = [c.name for c in chroma_client.list_collections()]
    if "images" in collections:
        image_collection = chroma_client.get_collection("images")
    else:
        image_collection = chroma_client.create_collection(
            name="images", metadata={"hnsw:space": "cosine"}
        )

    print("✅ Collections initialisiert")

    return RAGComponents(
        text_embeddings, clip_model, llm, text_splitter,
        markitdown, chroma_client, text_collection, image_collection
    )

## 2.4 Text-Dokument hinzufügen

In [7]:
def add_text_document(components: RAGComponents, file_path: str) -> bool:
    """Fügt ein Text-Dokument zur Datenbank hinzu"""
    path = Path(file_path).absolute()

    # Duplikatsprüfung
    if components.text_collection.get(where={"source": str(path)})['ids']:
        print(f"⚠️ {path.name} bereits vorhanden")
        return False

    try:
        # Dokument mit MarkItDown konvertieren
        result = components.markitdown.convert(str(path))
        if not result or not result.text_content.strip():
            print(f"⚠️ {path.name} enthält keinen Text")
            return False

        # Text in Chunks aufteilen
        chunks = components.text_splitter.split_text(result.text_content)
        documents = [
            Document(
                page_content=chunk.strip(),
                metadata={"source": str(path), "filename": path.name, "chunk_id": i}
            ) for i, chunk in enumerate(chunks) if chunk.strip()
        ]

        # Zur Datenbank hinzufügen
        if documents:
            components.text_collection.add_documents(documents)
            print(f"✅ {len(documents)} Chunks von '{path.name}' hinzugefügt")
            return True

    except Exception as e:
        print(f"❌ Fehler bei {path.name}: {e}")

    return False

## 2.5 Bild hinzufügen

In [8]:
def add_image(components: RAGComponents, image_path: str, description: str = "") -> bool:
    """Fügt ein Bild zur Datenbank hinzu"""
    path = Path(image_path).absolute()

    if not path.exists():
        print(f"❌ Bild nicht gefunden: {path}")
        return False

    # Duplikatsprüfung
    if components.image_collection.get(where={"source": str(path)})['ids']:
        print(f"⚠️ Bild bereits vorhanden: {path.name}")
        return False

    try:
        # Bild laden und Embedding erstellen
        image = Image.open(path).convert('RGB')
        embedding = components.clip_model.encode(image).tolist()
        print(f"🖼️ Bild-Embedding erstellt für {path.name}")

        # Metadaten vorbereiten
        doc_id = f"img_{uuid.uuid4().hex[:8]}_{path.name}"
        content = f"Bild: {path.name}"
        if description.strip():
            content += f" - {description.strip()}"

        # Zur Datenbank hinzufügen
        components.image_collection.add(
            ids=[doc_id],
            embeddings=[embedding],
            documents=[content],
            metadatas=[{
                "source": str(path),
                "filename": path.name,
                "description": description.strip()
            }]
        )

        print(f"✅ Bild '{path.name}' hinzugefügt")
        return True

    except Exception as e:
        print(f"❌ Fehler bei Bild {path.name}: {e}")
        return False

## 2.6 Verzeichnis verarbeiten

In [9]:
def process_directory(components: RAGComponents, directory: str, include_images: bool = True) -> Dict[str, int]:
    """Verarbeitet alle Dateien in einem Verzeichnis"""
    dir_path = Path(directory)
    if not dir_path.exists():
        print(f"❌ Verzeichnis nicht gefunden: {directory}")
        return {"texts": 0, "images": 0}

    # Unterstützte Dateitypen
    text_extensions = {'.pdf', '.docx', '.txt', '.md', '.html'}
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp'}

    # Dateien sammeln
    text_files = [f for f in dir_path.rglob("*") if f.suffix.lower() in text_extensions]
    image_files = [f for f in dir_path.rglob("*") if f.suffix.lower() in image_extensions] if include_images else []

    print(f"📊 Gefunden: {len(text_files)} Text-Dateien, {len(image_files)} Bilder")

    # Text-Dateien verarbeiten
    text_count = 0
    for file_path in text_files:
        print(f"📄 {file_path.name}")
        if add_text_document(components, str(file_path)):
            text_count += 1

    # Bild-Dateien verarbeiten
    image_count = 0
    for img_path in image_files:
        print(f"🖼️ {img_path.name}")
        # Automatische Beschreibung aus Dateiname
        description = img_path.stem.replace('_', ' ').replace('-', ' ')
        if add_image(components, str(img_path), description):
            image_count += 1

    return {"texts": text_count, "images": image_count}

## 2.7 Text-Suche

In [10]:
def search_texts(components: RAGComponents, query: str, k: int = 3, config: RAGConfig = RAGConfig()) -> str:
    """Durchsucht Text-Dokumente"""
    if not components.text_collection.get()['ids']:
        return "❌ Keine Text-Dokumente gefunden"

    # Ähnlichkeitssuche durchführen
    docs_with_scores = components.text_collection.similarity_search_with_score(query, k=k*2)
    if not docs_with_scores:
        return "❌ Keine relevanten Dokumente gefunden"

    # Nach Relevanz-Schwellenwert filtern
    relevant_docs = [(doc, score) for doc, score in docs_with_scores[:k]
                     if score < config.text_threshold]

    if not relevant_docs:
        return "❌ Keine ausreichend relevanten Dokumente gefunden"

    # Kontext für LLM zusammenstellen
    context = "\\n\\n---\\n\\n".join([doc.page_content for doc, _ in relevant_docs])
    sources = [{"filename": doc.metadata.get("filename", "Unbekannt"), "score": round(score, 3)}
               for doc, score in relevant_docs]

    # LLM-Antwort generieren
    prompt = f"""Beantworte die Frage präzise basierend auf dem Kontext.

KONTEXT:
{context}

FRAGE: {query}

ANTWORT:"""

    response = components.llm.invoke(prompt).content
    source_text = f"\\n\\n📚 Quellen ({len(sources)}): " + "\\n".join([
        f"   • {src['filename']} (Score: {src['score']})" for src in sources
    ])

    return f"{response}{source_text}"

## 2.8 Bild-Suche

In [11]:
def search_images(components: RAGComponents, query: str, k: int = 3, config: RAGConfig = RAGConfig()) -> List[Dict[str, Any]]:
    """Durchsucht Bilder mit Text-Query"""
    if components.image_collection.count() == 0:
        return []

    # Text-Query in Bild-Embedding-Raum umwandeln
    query_embedding = components.clip_model.encode(query).tolist()

    # Suche in Bild-Collection
    results = components.image_collection.query(
        query_embeddings=[query_embedding],
        n_results=min(k*2, components.image_collection.count()),
        include=['documents', 'metadatas', 'distances']
    )

    if not results['ids'][0]:
        return []

    # Ergebnisse filtern und formatieren
    return [
        {
            "filename": metadata.get("filename", "Unbekannt"),
            "path": metadata.get("source", ""),
            "description": metadata.get("description", ""),
            "similarity": round(max(0, 1 - distance), 3)
        }
        for distance, metadata in zip(results['distances'][0], results['metadatas'][0])
        if distance < config.image_threshold
    ]

## 2.9 Multimodale Suche

In [12]:
def multimodal_search(components: RAGComponents, query: str, k_text: int = 3, k_images: int = 3) -> str:
    """Führt multimodale Suche durch (Text + Bilder)"""
    mprint(f"## 🔍 Multimodale Suche: {query}")

    # Parallele Suche in beiden Modalitäten
    text_results = search_texts(components, query, k_text)
    image_results = search_images(components, query, k_images)

    # Ergebnisse zusammenfassen
    result = f"### 📄 TEXT-ERGEBNISSE:\n{text_results}\n\n"

    if image_results:
        result += f"### 🖼️ BILD-ERGEBNISSE ({len(image_results)} gefunden):\n"
        for i, img in enumerate(image_results, 1):
            result += f"   {i}. {img['filename']} (Ähnlichkeit: {img['similarity']})\n"
            if img['description']:
                result += f"      📝 {img['description']}\n"
    else:
        result += "🖼️ Keine relevanten Bilder gefunden.\n"

    return result

## 2.10 Hilfsfunktionen

In [13]:
def get_system_status(components: RAGComponents) -> Dict[str, Any]:
    """Gibt detaillierten System-Status zurück"""
    text_data = components.text_collection.get()
    text_count = len(text_data['ids'])
    image_count = components.image_collection.count()

    print(f"📊 Status: {text_count} Text-Chunks, {image_count} Bilder")

    return {
        "text_chunks": text_count,
        "images": image_count,
        "total_documents": text_count + image_count
    }

def cleanup_database(db_path: str = './multimodal_rag_db'):
    """Löscht die Datenbank komplett"""
    import shutil
    if Path(db_path).exists():
        shutil.rmtree(db_path)
        print(f"🗑️ Datenbank gelöscht: {db_path}")

# 3 | Hands-On: Multimodales RAG
---


## 3.1 Alte Datenbank löschen

In [14]:
# Aufräumen vor Neustart
cleanup_database('./multimodal_rag_db')

## 3.2 System initialisieren

In [15]:
# Konfiguration erstellen
config = RAGConfig(db_path='./multimodal_rag_db')

# RAG-System initialisieren
rag_components = init_rag_system(config)

🚀 Initialisiere RAG-System in ./multimodal_rag_db
✅ OpenAI Text-Embeddings initialisiert
🖼️ Lade CLIP-Modell...


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ CLIP-Modell geladen
✅ Collections initialisiert


## 3.3 Dokumente verarbeiten

In [16]:
# Alle Dateien im files-Verzeichnis verarbeiten
results = process_directory(rag_components, './files', include_images=True)

print(f"\\n📊 Verarbeitungsergebnis:")
print(f"   📄 Text-Dokumente: {results['texts']}")
print(f"   🖼️ Bilder: {results['images']}")
print(f"   📦 Gesamt: {results['texts'] + results['images']}")

📊 Gefunden: 4 Text-Dateien, 2 Bilder
📄 biografien_1.txt
✅ 32 Chunks von 'biografien_1.txt' hinzugefügt
📄 biografien_3.pdf
✅ 68 Chunks von 'biografien_3.pdf' hinzugefügt
📄 biografien_2.md
✅ 44 Chunks von 'biografien_2.md' hinzugefügt
📄 biografien_4.docx
✅ 63 Chunks von 'biografien_4.docx' hinzugefügt
🖼️ Apfel.png
❌ Fehler bei Bild Apfel.png: cannot identify image file '/content/files/Apfel.png'
🖼️ a_retro-futuristic_robot_dall_e.jpg
🖼️ Bild-Embedding erstellt für a_retro-futuristic_robot_dall_e.jpg
✅ Bild 'a_retro-futuristic_robot_dall_e.jpg' hinzugefügt
\n📊 Verarbeitungsergebnis:
   📄 Text-Dokumente: 4
   🖼️ Bilder: 1
   📦 Gesamt: 5


## 3.4 System-Status anzeigen

In [17]:
# Aktueller Status der Datenbank
status = get_system_status(rag_components)
print(f"\\nDatenbank enthält:")
print(f"   📄 {status['text_chunks']} Text-Chunks")
print(f"   🖼️ {status['images']} Bilder")
print(f"   📦 {status['total_documents']} Dokumente insgesamt")

📊 Status: 207 Text-Chunks, 1 Bilder
\nDatenbank enthält:
   📄 207 Text-Chunks
   🖼️ 1 Bilder
   📦 208 Dokumente insgesamt


## 3.5 Text-Suche testen

In [18]:
# Reine Text-Suche
query = "Wer ist Thoren Navarro?"
result = search_texts(rag_components, query)

mprint(f"### 🔍 TEXT-SUCHE")
mprint(f"**Query:** {query}")
mprint("---")
mprint(result)

### 🔍 TEXT-SUCHE

**Query:** Wer ist Thoren Navarro?

---

Thoren Navarro ist ein Unterwasserarchäologe und Klimaaktivist, der spezialisierte Tauchroboter entwickelt hat, um versunkene antike Städte zu kartieren und zu konservieren.\n\n📚 Quellen (1):    • biografien_2.md (Score: 0.761)

## 3.6 Bild-Suche testen

In [19]:
# Reine Bild-Suche
query = "Roboter"
images = search_images(rag_components, query)

mprint(f"### 🖼️ BILD-SUCHE")
mprint(f"**Query:** {query}")
mprint("---")

if images:
    mprint(f"**{len(images)} Bilder gefunden:**")
    for i, img in enumerate(images, 1):
        mprint(f"{i}. **{img['filename']}** (Ähnlichkeit: {img['similarity']})")
        if img['description']:
            mprint(f"   📝 *{img['description']}*")
else:
    mprint("Keine relevanten Bilder gefunden.")

### 🖼️ BILD-SUCHE

**Query:** Roboter

---

**1 Bilder gefunden:**

1. **a_retro-futuristic_robot_dall_e.jpg** (Ähnlichkeit: 0.272)

   📝 *a retro futuristic robot dall e*

## 3.7 Multimodale Suche testen

In [20]:
# Kombinierte Text- und Bild-Suche
test_queries = [
    "Wer ist Thoren Navarro?",
    "Zeige mir Bilder von Robotern",
    "Was sind KI-Modelle?",
    "Futuristische Technologien"
]

for query in test_queries:
    print(f"\\n{'='*60}")
    mprint(f"## 🔍 MULTIMODALE SUCHE: {query}")
    mprint('---')

    result = multimodal_search(rag_components, query, k_text=2, k_images=2)
    mprint(result)

\n============================================================


## 🔍 MULTIMODALE SUCHE: Wer ist Thoren Navarro?

---

## 🔍 Multimodale Suche: Wer ist Thoren Navarro?

### 📄 TEXT-ERGEBNISSE:
Thoren Navarro ist ein Unterwasserarchäologe und Klimaaktivist, der spezialisierte Tauchroboter entwickelt hat, um versunkene antike Städte zu kartieren und zu konservieren.\n\n📚 Quellen (1):    • biografien_2.md (Score: 0.761)

🖼️ Keine relevanten Bilder gefunden.


...

\n============================================================


## 🔍 MULTIMODALE SUCHE: Futuristische Technologien

---

## 🔍 Multimodale Suche: Futuristische Technologien

### 📄 TEXT-ERGEBNISSE:
Futuristische Technologien beziehen sich auf innovative Entwicklungen, die das Potenzial haben, die Zukunft zu gestalten, insbesondere in Bereichen wie Kommunikation, Medizin, Umwelt und Kunst. In diesem Kontext könnten sie dazu beitragen, das Bewusstsein für Vergänglichkeit zu schärfen und gleichzeitig Traditionen, wie die japanische Musik, in therapeutischen Anwendungen zu integrieren.\n\n📚 Quellen (2):    • biografien_3.pdf (Score: 0.991)\n   • biografien_1.txt (Score: 1.0)

### 🖼️ BILD-ERGEBNISSE (1 gefunden):
   1. a_retro-futuristic_robot_dall_e.jpg (Ähnlichkeit: 0.232)
      📝 a retro futuristic robot dall e


# 4 | Erweiterte Funktionen
---



## 4.1 Batch-Verarbeitung

In [21]:
def process_multiple_directories(components: RAGComponents, directories: List[str]) -> Dict[str, Any]:
    """Verarbeitet mehrere Verzeichnisse in einem Durchgang"""
    total_results = {"texts": 0, "images": 0}

    for directory in directories:
        print(f"\\n📁 Verarbeite Verzeichnis: {directory}")
        results = process_directory(components, directory, include_images=True)
        total_results["texts"] += results["texts"]
        total_results["images"] += results["images"]

    return total_results

# Usage
directories = ['./docs', './images', './pdfs']
# total = process_multiple_directories(rag_components, directories)

## 4.2 Erweiterte Suchoptionen

In [22]:
def advanced_search(components: RAGComponents, query: str, filters: Dict[str, Any] = None) -> Dict[str, Any]:
    """Erweiterte Suche mit Filtern"""
    filters = filters or {}

    # Text-Suche mit optionalen Filtern
    text_results = []
    if not filters.get('images_only', False):
        text_results = search_texts(components, query, k=filters.get('k_text', 3))

    # Bild-Suche mit optionalen Filtern
    image_results = []
    if not filters.get('text_only', False):
        image_results = search_images(components, query, k=filters.get('k_images', 3))

    return {
        "query": query,
        "text_results": text_results,
        "image_results": image_results,
        "total_found": len(text_results) + len(image_results)
    }

# Usage Examples
# result = advanced_search(rag_components, "KI", {"text_only": True})
# result = advanced_search(rag_components, "Roboter", {"images_only": True})

## 4.3 Performance-Monitoring

In [23]:
import time
from functools import wraps

def measure_time(func):
    """Decorator für Performance-Messung"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        print(f"⏱️ {func.__name__} dauerte {duration:.2f}s")
        return result
    return wrapper

@measure_time
def timed_multimodal_search(components: RAGComponents, query: str) -> str:
    """Multimodale Suche mit Zeitmessung"""
    return multimodal_search(components, query)

# Usage
result = timed_multimodal_search(rag_components, "Test Query")

## 🔍 Multimodale Suche: Test Query

⏱️ timed_multimodal_search dauerte 0.90s


# 5 | Performance Test
---


In [24]:
import time
from typing import Callable

def benchmark_function(func: Callable, *args, iterations: int = 10) -> Dict[str, float]:
    """Benchmarkt eine Funktion über mehrere Iterationen"""
    times = []

    for _ in range(iterations):
        start = time.time()
        func(*args)
        end = time.time()
        times.append(end - start)

    return {
        "avg_time": sum(times) / len(times),
        "min_time": min(times),
        "max_time": max(times),
        "total_time": sum(times)
    }

# Usage Example
benchmark_result = benchmark_function(search_texts, rag_components, "test query")
print(f"Durchschnittliche Suchzeit: {benchmark_result['avg_time']:.3f}s")

Durchschnittliche Suchzeit: 0.346s


# 6 | Deployment
---



## 6.1 FastAPI Integration


<img src="https://raw.githubusercontent.com/ralf-42/Image/main/under_construction_dall_e_klein.png" class="logo" width="500"/>



In [25]:
import nest_asyncio
nest_asyncio.apply()

from contextlib import asynccontextmanager
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

# Global components (in production: use dependency injection)
rag_components = None

class SearchRequest(BaseModel):
    query: str
    k_text: Optional[int] = 3
    k_images: Optional[int] = 3
    search_type: Optional[str] = "multimodal"  # "text", "images", "multimodal"

class SearchResponse(BaseModel):
    query: str
    result: str
    search_type: str
    processing_time: float

@asynccontextmanager
async def lifespan(app: FastAPI):
    """Initialize RAG system on startup"""
    global rag_components
    config = RAGConfig(db_path="./production_rag_db")
    rag_components = init_rag_system(config)

    yield

app = FastAPI(title="Funktionales RAG API", lifespan=lifespan)

@app.post("/search", response_model=SearchResponse)
async def search_endpoint(request: SearchRequest):
    """API Endpoint für Suchen"""
    if not rag_components:
        raise HTTPException(status_code=500, detail="RAG system not initialized")

    start_time = time.time()

    try:
        if request.search_type == "text":
            result = search_texts(rag_components, request.query, request.k_text)
        elif request.search_type == "images":
            images = search_images(rag_components, request.query, request.k_images)
            result = f"Found {len(images)} images: " + ", ".join([img['filename'] for img in images])
        else:  # multimodal
            result = multimodal_search(rag_components, request.query, request.k_text, request.k_images)

        processing_time = time.time() - start_time

        return SearchResponse(
            query=request.query,
            result=result,
            search_type=request.search_type,
            processing_time=processing_time
        )

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Search failed: {str(e)}")

@app.post("/add-document")
async def add_document_endpoint(file_path: str):
    """API Endpoint zum Hinzufügen von Dokumenten"""
    if not rag_components:
        raise HTTPException(status_code=500, detail="RAG system not initialized")

    success = add_text_document(rag_components, file_path)
    return {"success": success, "file_path": file_path}


# if __name__ == "__main__":
#     uvicorn.run(app, host="0.0.0.0", port=8000)

## 6.2 Gradio Interface

In [ ]:
import gradio as gr

def create_gradio_interface(components: RAGComponents) -> gr.Blocks:
    """Erstellt eine benutzerfreundliche Web-Oberfläche"""

    with gr.Blocks(title="Funktionales Multimodales RAG") as interface:
        gr.HTML("<h1>🔍 Multimodales RAG System</h1>")

        with gr.Tab("Suchen"):
            with gr.Row():
                with gr.Column(scale=2):
                    query_input = gr.Textbox(
                        label="Ihre Frage",
                        placeholder="z.B. 'Wer ist Thoren Navarro?' oder 'Zeige mir Roboter-Bilder'",
                        lines=3
                    )

                    with gr.Row():
                        search_btn = gr.Button("🔍 Suchen", variant="primary")
                        clear_btn = gr.Button("🗑️ Löschen")

                with gr.Column(scale=1):
                    search_type = gr.Radio(
                        choices=["multimodal", "text", "images"],
                        value="multimodal",
                        label="Suchtyp"
                    )
                    k_text = gr.Slider(1, 10, 3, label="Anzahl Text-Ergebnisse")
                    k_images = gr.Slider(1, 10, 3, label="Anzahl Bild-Ergebnisse")

            result_output = gr.Markdown(label="Ergebnisse")

            def perform_search(query, stype, kt, ki):
                if not query.strip():
                    return "❌ Bitte geben Sie eine Frage ein."

                try:
                    if stype == "text":
                        return search_texts(components, query, int(kt))
                    elif stype == "images":
                        images = search_images(components, query, int(ki))
                        if images:
                            result = f"### 🖼️ {len(images)} Bilder gefunden:\n"
                            for i, img in enumerate(images, 1):
                                result += f"{i}. **{img['filename']}** (Ähnlichkeit: {img['similarity']})\n"
                                if img['description']:
                                    result += f"   📝 *{img['description']}*\n"
                            return result
                        else:
                            return "❌ Keine relevanten Bilder gefunden."
                    else:  # multimodal
                        return multimodal_search(components, query, int(kt), int(ki))

                except Exception as e:
                    return f"❌ Fehler bei der Suche: {str(e)}"

            search_btn.click(
                perform_search,
                inputs=[query_input, search_type, k_text, k_images],
                outputs=result_output
            )

            clear_btn.click(
                lambda: ("", ""),
                outputs=[query_input, result_output]
            )

        with gr.Tab("System Status"):
            status_btn = gr.Button("📊 Status aktualisieren")
            status_output = gr.JSON(label="System-Informationen")

            def update_status():
                return get_system_status(components)

            status_btn.click(update_status, outputs=status_output)

        with gr.Tab("Dokumente hinzufügen"):
            with gr.Row():
                file_input = gr.File(
                    label="Dokument hochladen",
                    file_types=[".txt", ".md", ".pdf", ".docx"]
                )
                upload_btn = gr.Button("📄 Hinzufügen")

            upload_output = gr.Textbox(label="Upload-Status")

            def upload_document(file):
                if file is None:
                    return "❌ Keine Datei ausgewählt."

                try:
                    success = add_text_document(components, file.name)
                    if success:
                        return f"✅ Dokument '{file.name}' erfolgreich hinzugefügt."
                    else:
                        return f"⚠️ Dokument '{file.name}' konnte nicht hinzugefügt werden."
                except Exception as e:
                    return f"❌ Fehler beim Upload: {str(e)}"

            upload_btn.click(upload_document, inputs=file_input, outputs=upload_output)

    return interface

# Interface starten
interface = create_gradio_interface(rag_components)
interface.launch(share=True)

Exception in thread Thread-9 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/events.py", line 681, in get_event_loop
    raise RuntimeError('There is no current event loop in thread %r.'
RuntimeError: There is no current 

# A | Aufgaben
---

## A.1 Erweiterte Suchfunktion



Implementieren Sie eine `similarity_search` Funktion, die Dokumente nach Ähnlichkeit zu einem gegebenen Text-Embedding findet:

In [ ]:
def similarity_search(components: RAGComponents,
                     target_embedding: List[float],
                     k: int = 5,
                     threshold: float = 0.8) -> List[Dict[str, Any]]:
    """
    Findet ähnliche Dokumente basierend auf einem Embedding

    Args:
        components: RAG-System-Komponenten
        target_embedding: Ziel-Embedding für Vergleich
        k: Anzahl der zurückzugebenden Ergebnisse
        threshold: Mindest-Ähnlichkeit (0-1)

    Returns:
        Liste von ähnlichen Dokumenten mit Metadaten
    """
    # TODO: Implementierung
    pass

# Test der Funktion
# text_embedding = components.text_embeddings.embed_query("Künstliche Intelligenz")
# similar_docs = similarity_search(rag_components, text_embedding, k=3, threshold=0.7)

## A.2 Batch-Embedding



Erstellen Sie eine Funktion für effiziente Batch-Verarbeitung von Embeddings:

In [ ]:
from typing import Iterator

def batch_create_embeddings(components: RAGComponents,
                           texts: List[str],
                           batch_size: int = 10) -> Iterator[List[List[float]]]:
    """
    Erstellt Embeddings in Batches für bessere Performance

    Args:
        components: RAG-System-Komponenten
        texts: Liste von Texten
        batch_size: Größe der Batches

    Yields:
        Batches von Embeddings
    """
    # TODO: Implementierung mit Batch-Processing
    pass

# Usage Example
# texts = ["Text 1", "Text 2", "Text 3", ...]
# for embedding_batch in batch_create_embeddings(rag_components, texts, batch_size=5):
#     print(f"Verarbeitet {len(embedding_batch)} Embeddings")

## A.3 Evaluations-Framework



Entwickeln Sie ein System zur Bewertung der Suchqualität:

In [ ]:
@dataclass
class EvaluationResult:
    """Ergebnis einer Evaluation"""
    precision: float
    recall: float
    f1_score: float
    avg_response_time: float
    total_queries: int

def evaluate_search_quality(components: RAGComponents,
                          test_queries: List[str],
                          expected_results: List[List[str]]) -> EvaluationResult:
    """
    Evaluiert die Qualität des Suchsystems

    Args:
        components: RAG-System-Komponenten
        test_queries: Liste von Test-Anfragen
        expected_results: Erwartete Ergebnisse pro Anfrage

    Returns:
        Evaluations-Ergebnis mit Metriken
    """
    # TODO: Implementierung der Evaluation
    pass

# Test Data
test_data = [
    ("Wer ist Thoren Navarro?", ["biografien_2.md"]),
    ("Roboter Bilder", ["robot_image.jpg", "cyborg.png"]),
    # ... weitere Test-Fälle
]

# Usage
# queries, expected = zip(*test_data)
# results = evaluate_search_quality(rag_components, queries, expected)
# print(f"F1-Score: {results.f1_score:.3f}")